# MASH workflow
This note book document the process of meta analysis. 

The output of QTL association will firstly be grouped based on whether they could be conducted meta analysis according to a METAL_theme_str input.

Those that were grouped together will be fed into METAL and generate 1 sumstat for each chromosome. These sumstat will then be treated as the sumstat of a new theme, connected by -.

all the non_METALED and the new theme will be partioned from chromosome into genes, and then merged together.

For example, in a three theme study, AC, DLPFC, and PCC
a METAL_theme_str of "AC,DLPFC" indicates AC and DLPFC are to be METALed. Their METAL result will be named as AC-DLPFC

Then the AC-DLPFC and the PCC will each be partitioned and then merged together for downstream analysis.

AC and DLPFC will not be analyzed individually anymore.

In [ ]:
[global]
import os
# Work directory & output directory
parameter: wd = "./"
# The filename name for output data
parameter: container = 'gaow/twas'
# name for the analysis output
parameter: name = 'ROSMAP'
# For cluster jobs, number commands to run per job
parameter: job_size = 1
# Wall clock time expected
parameter: walltime = "5h"
# Memory expected
parameter: mem = "16G"
# Number of threads
parameter: numThreads = 20
# Diretory to the executable
parameter: exe_dir = path("~/GIT/ADSPFG-xQTL/workflow")
# yml template
parameter: yml = f'{exe_dir:d}/code/csg.yml'
# queue for analysis
parameter: queue = "csg"
# Number of submission
parameter: J = 200
# Mash Options
parameter: vhat = 'simple'
parameter: region_list = path
import csv
import pandas as pd
## A multi column file, each row is 1 chr, first col is chr, each subsqeunt a theme. each cell is the path to a sumstat file, which with following column format: variant_id, alt, ref, pval, beta, se ; 
parameter: sumstat_list_path = path
sumstat_list =  pd.read_csv(sumstat_list_path,delimiter="\t")
## A comma sep str that indicates what are the themes that goes into metal
parameter: METAL_theme_str = "."


## Retain the chr column as well
METAL_theme = METAL_theme_str.split(",")
METAL_theme_prefix = "-".join(METAL_theme)
METAL_theme.append("chr")
METAL_list = sumstat_list[METAL_theme].values.tolist()
Non_METAL_list= sumstat_list.drop(METAL_theme_str.split(","),axis = 1)
chrom = sumstat_list["chr"].values.tolist()
METALed_sumstat_list = Non_METAL_list.assign(**{METAL_theme_prefix : [f'{wd}/METAL/{METAL_theme_prefix}.chr{x}.METAL.txt' for x in chrom ]})
Theme = METALed_sumstat_list.drop(["chr"],axis = 1).columns.values.tolist()
Theme_list = pd.DataFrame({"#Theme" : [f'{wd}/sumstat/{x}' for x in Theme]})
Theme_prefix = "_".join(Theme)


## Extract effect


In [ ]:
[extract_effects]
input: output_from("merge_and_alleleQC")["merged_analysis_unit"]
output: extracted_effect = f'{wd}/sumstat/{Theme_prefix}.rds'
bash: expand = "$[ ]", stderr = f'{_output[0]}.stderr', stdout = f'{_output[0]}.stdout'
            sos run $[exe_dir]/Meta_Analysis/Reformatting/Signal_Extraction.ipynb extract_effects \
            --cwd $[wd]/sumstat/ \
            --container $[container] \
            --name $[Theme_prefix] \
            --analysis_units $[_input] \
            -J $[J] -q $[queue] -c $[yml]

## Factor analysis and MASH Model

In [ ]:
[MASH_FLASH]
parameter: effect_model = 'EZ'
parameter: mixture_components = ['flash', 'flash_nonneg', 'pca',"canonical"]
input: output_from("extract_effects")["extracted_effect"]
output: MASH_model = f"{wd}/sumstat/{Theme_prefix}.{effect_model}.V_{vhat}.mash_model.rds",
        resid_corr = f"{wd}/sumstat/{Theme_prefix}.{effect_model}.V_{vhat}.rds",
        flash_output = [f"{wd}/sumstat/{Theme_prefix}.{m}.rds" for m in mixture_components]
bash: expand = "$[ ]", stderr = f'{_output[0]}.stderr', stdout = f'{_output[0]}.stdout'
            sos run $[exe_dir]/Meta_Analysis/MASH/mashr_flashr_workflow.ipynb mash \
            --cwd $[wd]/sumstat/ \
            --container $[container] \
            --effect_model $[effect_model] \
            --vhat $[vhat] \
            --output_prefix $[Theme_prefix] \
            --data $[_input] \
            -J $[J] -q $[queue] -c $[yml]

## MASH Posterior

In [ ]:
[MASH_posterior]
parameter: effect_model = 'EZ'
parameter: mixture_components = ['flash', 'flash_nonneg', 'pca',"canonical"]
analysis_units = f'{wd}/sumstat/merged_analysis_unit.txt'
regions = [x.replace("\"","").strip().split() for x in open(analysis_units).readlines() if x.strip() and not x.strip().startswith('#')]
gene = [x[0] for x in regions]
input: output_from("MASH_FLASH")["MASH_model"], regions
output: mash_output_list = f'{wd}/sumstat/mash_output_list'
bash: expand = "$[ ]", stderr = f'{_output[0]}.stderr', stdout = f'{_output[0]}.stdout'
            sos run $[exe_dir]/Meta_Analysis/MASH/mash_posterior.ipynb  posterior \
            --cwd $[wd]/sumstat/ \
            --container $[container] \
            --mash_model $[_input[0]] \
            --vhat $[vhat] \
            --output_prefix $[Theme_prefix] \
            --posterior_input  $[" ".join(gene)] \
            -J $[J] -q $[queue] -c $[yml]

## Mixture Prior

In [ ]:
[Mixture_prior]
parameter: effect_model = 'EZ'
parameter: mixture_components = ['flash', 'flash_nonneg', 'pca',"canonical"]
parameter: mixture_prior_method = "ed_bovy"
input: output_from("MASH_FLASH")["MASH_model"], output_from("extract_effects")["extracted_effect"]
analysis_units = f'{wd}/sumstat/merged_analysis_unit.txt'
regions = [x.replace("\"","").strip().split() for x in open(analysis_units).readlines() if x.strip() and not x.strip().startswith('#')]
output: mixture_prior = f'{wd}/../Fine_Mapping/Mixture_Prior/{Theme_prefix}.{mixture_prior_method}.V_{vhat}.rds'
bash: expand = "$[ ]", stderr = f'{_output[0]}.stderr', stdout = f'{_output[0]}.stdout'
            sos run $[exe_dir]/Meta_Analysis/MASH/mixture_prior.ipynb $[mixture_prior_method] \
            --cwd $[wd] \
            --container $[container] \
            --name $[Theme_prefix] \
            --vhat $[vhat] \
            --data  $[_input[1]] \
            --name $[Theme_prefix] \
            -J $[J] -q $[queue] -c $[yml]

## RDS to VCF

In [ ]:
[RDS2VCF]
parameter: data_dir = path("/")
input: output_from("MASH_posterior")["mash_output_list"]
output: f'{wd}/mash_vcf/vcf_output_list.txt'
bash: expand = "$[ ]", stderr = f'{_output[0]}.stderr', stdout = f'{_output[0]}.stdout'
        sos run $[exe_dir]/Meta_Analysis/Reformatting/RDS_to_vcf.ipynb rds_to_vcf \
            --wd $[wd]/ \
            --container $[container] \
            --name $[Theme_prefix] \
            --analysis_units $[_input] \
            --data_dir $[data_dir]  \
            -J $[J] -q $[queue] -c $[yml]